In [ ]:
import numpy as np

In [ ]:
raw_averaged_file_path = "/nsls2/data/projects/ldrd-22-031-blopt/raw_averaged.npy"

In [ ]:
%time raw_avg_images = np.load(raw_averaged_file_path).astype(np.float32)

In [ ]:
print(f"raw_avg_images.shape: {raw_avg_images.shape}")
raw_avg_images[0]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3)
axs[0].imshow(raw_avg_images[0, :, :])
axs[0].set_title("raw averaged image 0")
axs[1].hist(raw_avg_images[0, :, :].flatten(), log=True, bins=20)
axs[2].hist(raw_avg_images[::100].flatten(), bins=20, log=True)
fig.show()

In [ ]:
pip install opencv-python

In [ ]:
import cv2

In [ ]:
resized_raw_avg_images = np.zeros((raw_avg_images.shape[0], 16, 16))
for i in range(resized_raw_avg_images.shape[0]):
    resized_raw_avg_images[i, :, :] = cv2.resize(raw_avg_images[i, :, :], resized_raw_avg_images.shape[1:])

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3)
axs[0].imshow(raw_avg_images[0])
axs[0].set_title("raw averaged image 0")
axs[1].imshow(resized_raw_avg_images[0])
axs[2].hist(resized_raw_avg_images.flatten(), log=True)
fig.show()

In [ ]:
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
pip install torchinfo

In [ ]:
import torch
import torch.nn as nn
from torchinfo import summary

In [ ]:
def build_beamline_model(parameter_count):
    # take four parameters and expand them to a larger layer
    beamline_middle = nn.Sequential(
        nn.Linear(parameter_count, 8),
        nn.ReLU(),
        nn.Linear(8, 64),
        nn.ReLU(),
        nn.Linear(64, 256),  # for 16x16 filter
        # nn.ReLU()
    )
    beamline_up = nn.Sequential(
        nn.Conv2d(
            in_channels=1,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=32,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=16,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=8,
            out_channels=1,
            kernel_size=3,
            stride=1,
            padding=1
        ),
    )

    class BL(nn.Module):
        def __init__(self):
            super().__init__()

            self.beamline_middle = beamline_middle
            self.beamline_up = beamline_up

        def forward(self, beamline_parameters):
            batch_count = beamline_parameters.shape[0]

            beamline_parameters_embedding = self.beamline_middle(beamline_parameters)

            beamline_parameters_embedding_2d = beamline_parameters_embedding.reshape(batch_count, -1, 16, 16)
            
            image = self.beamline_up(beamline_parameters_embedding_2d)

            return image
    
    return BL()


In [ ]:
b_m = build_beamline_model(4)

In [ ]:
summary(
    b_m, 
    input_data=torch.ones(2, 1, 4),
    col_names=("input_size", "output_size", "num_params")
)

In [ ]:
from databroker import Broker
db = Broker.named("tes")

In [ ]:
def get_beamline_parameters(db):
    hdr = db[9585]
    the_data = hdr.table()
    the_parameters = the_data[['kbh_ush','kbh_dsh','kbv_ush','kbv_dsh']]
    return the_parameters

In [ ]:
%time beamline_parameters = np.array(get_beamline_parameters(db), dtype=np.float32)

In [ ]:
print(f"beamline_parameters.shape: {beamline_parameters.shape}")

In [ ]:
# a class to interact with DataLoaders
class BeamlineDataset:
    def __init__(self, beamline_parameters, beam_images):
        """
          beamline_parameters  samples x 4       (4 motor positions)
          beam_images          samples x 16 x 16
        """
        self.beamline_parameters = np.array(beamline_parameters, dtype=np.float32)
        # expand the dimensions of beam_images to include a "channel" dimension
        self.beam_images = np.expand_dims(np.array(beam_images, dtype=np.float32), axis=1)
        self.beam_images_brightest_pixel = np.argmax(beam_images)

    def __getitem__(self, index):
        return self.beamline_parameters[index], self.beam_images[index]

    def __len__(self):
        return len(self.beamline_parameters)

In [ ]:
def build_beamline_dataset(beamline_parameters, beam_images, training_sample_count, testing_sample_count):
    shuffled_full_int_index = np.array(
        list(
            range(training_sample_count + testing_sample_count)
        ),
        dtype=int
    )
    np.random.shuffle(shuffled_full_int_index)

    training_int_index = shuffled_full_int_index[:training_sample_count]
    testing_int_index = shuffled_full_int_index[training_sample_count:training_sample_count+testing_sample_count]
    
    training_beamline_parameters = beamline_parameters[training_int_index]
    training_beam_images = beam_images[training_int_index]
    
    testing_beamline_parameters = beamline_parameters[testing_int_index]
    testing_beam_images = beam_images[testing_int_index]

    training_beamline_dataset = BeamlineDataset(
        beamline_parameters=training_beamline_parameters,
        beam_images=training_beam_images
    )

    print(f"len(training_beamline_dataset): {len(training_beamline_dataset)}")
    training_beamline_parameters_sample, training_beam_images_sample = training_beamline_dataset[1]
    print(f"training_beamline_parameters_sample.shape : {training_beamline_parameters_sample.shape}")
    print(f"training_beamline_parameters_sample.dtype : {training_beamline_parameters_sample.dtype}")
    print(f"training_beam_images_sample.shape : {training_beam_images_sample.shape}")
    print(f"training_beam_images_sample.dtype : {training_beam_images_sample.dtype}")
    
    testing_beamline_dataset = BeamlineDataset(
        beamline_parameters=testing_beamline_parameters,
        beam_images=testing_beam_images
    )
    
    return training_beamline_dataset, testing_beamline_dataset

build_beamline_dataset(
    beamline_parameters=beamline_parameters,
    beam_images=resized_raw_avg_images,
    training_sample_count=10,
    testing_sample_count=10
)

In [ ]:
def train(
    beamline_model,
    optimizer,
    loss_function,
    train_dataloader,
    test_dataloader,
    epoch_count
):
    
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    beamline_model.to(device)

    for epoch_i in range(epoch_count):
        training_loss = 0.0
        beamline_model.train()
        #for correct_squashed_circle_images, (circle_images, radius_scale_factors) in train_dataloader:
        for beamline_parameters, correct_beam_images in train_dataloader:
            optimizer.zero_grad()

            # torch calls circle_images 'inputs'
            correct_beam_images = correct_beam_images.to(device)
            #correct_squashed_circle_images = correct_squashed_circle_images.to(device)
            beamline_parameters = beamline_parameters.to(device)
            
            predicted_beam_images = beamline_model(
                beamline_parameters
            )

            loss = loss_function(
                predicted_beam_images,
                correct_beam_images
            )
            loss.backward()
            optimizer.step()

            training_loss += loss.data.item()

        training_loss /= len(train_dataloader.dataset)

        test_loss = 0.0
        beamline_model.eval()
        for beamline_parameters, correct_beam_images in test_dataloader:

            # torch calls circle_images 'inputs'
            #circle_images = circle_images.to(device)
            correct_beam_images = correct_beam_images.to(device)
            beamline_parameters = beamline_parameters.to(device)

            predicted_beam_images = beamline_model(
                beamline_parameters
            )

            loss = loss_function(
                predicted_beam_images,
                correct_beam_images
            )
            test_loss += loss.data.item()

        test_loss /= len(test_dataloader.dataset)

        print(
            f"Epoch: {epoch_i}, Training Loss: {training_loss:.5f}, Test Loss: {test_loss:.5f}"
        )


In [ ]:
training_dataset, testing_dataset = build_beamline_dataset(
    beamline_parameters=beamline_parameters,
    beam_images=resized_raw_avg_images,
    training_sample_count=40000,
    testing_sample_count=10000
)

In [ ]:
import torch.optim
from torch.utils.data import DataLoader

training_beamline_dataloader = DataLoader(
    training_dataset,
    batch_size=100,  # one batch must fit in the GPU memory
    shuffle=True
)

testing_beamline_dataloader = DataLoader(
    testing_dataset,
    batch_size=10
)

beamline_model = build_beamline_model(parameter_count=4)
train(
    beamline_model,
    torch.optim.Adam(beamline_model.parameters()),
    torch.nn.MSELoss(),
    training_beamline_dataloader,
    testing_beamline_dataloader,
    epoch_count=500
)


In [ ]:
# find the range of each beamline parameter
beamline_parameter_mins = np.amin(beamline_parameters, axis=0)
beamline_parameter_maxs = np.amax(beamline_parameters, axis=0)

print(f"beamline parameter mins: {beamline_parameter_mins}")
print(f"beamline parameter maxs: {beamline_parameter_maxs}")

In [ ]:
def predict_beam(beamline_model, beamline_parameters, kbh_ush, kbh_dsh, kbv_ush, kbv_dsh):
    
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    beamline_model.to(device)

    beamline_model.eval()

    beamline_parameters_for_prediction = np.array(
        [[kbh_ush, kbh_dsh, kbv_ush, kbv_dsh],],
    )
    print(f"beamline_parameters_for_prediction.shape: {beamline_parameters_for_prediction.shape}")

    # find the nearest real image
    m = np.sum(np.square(beamline_parameters_for_prediction - beamline_parameters))
    m_argmin = np.argmin(m)
    
    nearest_beamline_parameters = beamline_parameters[m_argmin, :]
    print(f"nearest beamline parameters: {nearest_beamline_parameters}")
    
    #beamline_parameters = beamline_parameters.to(device)
    predicted_beam_image = beamline_model(torch.Tensor(beamline_parameters_for_prediction)).detach().numpy()
    
    fig, axs = plt.subplots(nrows=1, ncols=4)
    axs[0].imshow(predicted_beam_image[0, 0, :, :])
    axs[0].set_title("predicted\nbeam\nimage")
    axs[1].hist(predicted_beam_image.flatten())
    axs[2].imshow(resized_raw_avg_images[m_argmin])
    axs[3].hist(resized_raw_avg_images[m_argmin].flatten())
    fig.show()

In [ ]:
predict_beam(beamline_model, beamline_parameters, 2.21212, 3.41414, 2.61616, 4.31313)

In [ ]:
np.argmax(raw_avg_images[:3, :, :], axis=0)